# Archived Hours Analysis

Looking at the current status of our archived hours for what we would have to do to get under the 4400 quota for our plan and not need to purchase an additional archived hours bundle. Run the "System session storage" report which I believe is the only one that shows whether a session is archived or not.

We want to check if a session is owned by someone at CCA too so we download student and employee data to get a list of all current usernames.

In [ ]:
#!/usr/bin/env bash
gsutil cp gs://integration-success/employee_data.json .
gsutil cp gs://integration-success/student_data.json .
jq -r ".Report_Entry[].username" employee_data.json > employee_usernames.txt
jq -r ".Report_Entry[].username" student_data.json > student_usernames.txt
cat employee_usernames.txt student_usernames.txt | sort | uniq > usernames.txt
# clean up files
rm employee_data.json student_data.json employee_usernames.txt student_usernames.txt

In [1]:
# read usernames line into list
with open('usernames.txt', 'r') as fh:
    usernames: list[str] = [line.strip() for line in fh.readlines()]
usernames[0:5]

['2012332', '2026537', '2028203', '2029180', '2029995']

In [2]:
import pandas as pd

# load the data
df = pd.read_csv(
    "systemsessionstorage_2020-08-08--2025-05-13.csv", index_col="Session ID"
)
# these three columns are datetimes
df["Start Time"] = pd.to_datetime(df["Start Time"])
df["Most Recent View Date"] = pd.to_datetime(
    df["Most Recent View Date"], errors="coerce"
)
df["Archived Date"] = pd.to_datetime(
    df["Archived Date"], errors="coerce"
)
df.info()

/var/folders/2r/18pvrjyn543fp646nfn5hv2h0000gp/T/ipykernel_54906/1744267190.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Start Time"] = pd.to_datetime(df["Start Time"])
/var/folders/2r/18pvrjyn543fp646nfn5hv2h0000gp/T/ipykernel_54906/1744267190.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Most Recent View Date"] = pd.to_datetime(


<class 'pandas.core.frame.DataFrame'>
Index: 39438 entries, f46327ca-bcba-4365-9ed5-ac12015850f2 to c62f2dbf-8f10-46ec-a06e-af6a01678cb6
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Session Name                     39438 non-null  object        
 1   Creator                          39438 non-null  object        
 2   Creator ID                       39438 non-null  object        
 3   Email                            39428 non-null  object        
 4   Start Time                       39438 non-null  datetime64[ns]
 5   Folder Name                      39438 non-null  object        
 6   Folder ID                        39438 non-null  object        
 7   Views and Downloads              39438 non-null  int64         
 8   Unique Viewers                   39438 non-null  int64         
 9   Minutes Delivered                39438 non-null  float64       
 1

In [3]:
# new column: whether Creator minus "unified/" is in the usernames list
df["Currently at CCA?"] = df["Creator"].str.replace("unified\\", "").isin(usernames)
df.head()

,Session Name,Creator,Creator ID,Email,Start Time,Folder Name,Folder ID,Views and Downloads,Unique Viewers,Minutes Delivered,...,MSO - Media Originals,MSO - Editor Metadata,MSO - Media Quality,MSO - Single Stream Viewing,Root Folder (Level 0),Subfolder (Level 1),Subfolder (Level 2),Subfolder (Level 3),Subfolder (Level 4),Currently at CCA?
Session ID,,,,,,,,,,,,,,,,,,,,,
f46327ca-bcba-4365-9ed5-ac12015850f2,2020-08-06 Moodle Getting Started,unified\bobbydeetz,1ed90e5b-7c4b-4ee3-8a8c-acf40183b181,bobbydeetz@cca.edu,2020-08-09 13:53:37,Moodle,af1ec353-b43a-446d-97c1-ac2700f9dd2d,10,8,128.9823,...,False,False,False,False,Libraries,Faculty Workshop Recordings,NaN,NaN,NaN,False
7989c38b-bc0a-4ef1-9fd5-ac1b01155441,test,unified\tlarson,ced1bf40-ab7b-4b70-a934-ac22015b536b,tlarson@cca.edu,2020-08-18 09:49:44,tlarson_796a9a8b-ef64-4115-9e14-ac1a0139eec4,675e91b8-3ec2-465c-b519-ac1a0139ef0e,1,1,0.2302,...,False,False,False,False,Users,Archive,NaN,NaN,NaN,False
6b2d72b4-189e-4c56-be4d-ac1b017fb23a,Test Video recorded from Panopto,unified\ephetteplace,661be145-86ec-4063-af5c-ac1d01714227,ephetteplace@cca.edu,2020-08-18 16:17:31,DEMO-2020-2021-Mobile-Testing: DEMO 2020 Model...,879c9e52-6861-43fc-935e-ac1b017070c0,6,4,1.0516,...,True,True,True,False,Moodle,2020FA,Metacourses,NaN,NaN,True
ea9c6103-a9a0-4436-a676-ac1b0180d453,openEQUELLA shared search bug,unified\ephetteplace,661be145-86ec-4063-af5c-ac1d01714227,ephetteplace@cca.edu,2020-08-18 16:21:07,DEMO-2020-2021-Mobile-Testing: DEMO 2020 Model...,879c9e52-6861-43fc-935e-ac1b017070c0,2,2,1.0363,...,True,True,True,False,Moodle,2020FA,Metacourses,NaN,NaN,True
1483b3c4-6cda-44db-b412-ac1d0138e81c,"2020-08-20 Int/Adv Moodle: Gradebook, Rubrics,...",unified\bobbydeetz,1ed90e5b-7c4b-4ee3-8a8c-acf40183b181,bobbydeetz@cca.edu,2020-08-20 10:07:19,Moodle,af1ec353-b43a-446d-97c1-ac2700f9dd2d,34,24,737.4727,...,False,False,False,False,Libraries,Faculty Workshop Recordings,NaN,NaN,NaN,False


In [4]:
# total hours

print('Total hours:', round(df["Session Length (Edited minutes)"].sum() / 60))
# archived hours
print('Archived hours:', round(df["Archived Hours (Unedited hours)"].sum()))
# archived hours owned by users no longer at CCA
hrs = df[df["Currently at CCA?"] == False]["Archived Hours (Unedited hours)"].sum()
print('Archived hours owned by users no longer at CCA:', round(hrs))

Total hours: 20953
Archived hours: 16280
Archived hours owned by users no longer at CCA: 2455


We have 8800 stored hours and 4400 archived so total hours has to get well below 13200 and archived has to get below 4400. We need to give ourselves breathing room here, too.

In [5]:
# unwatched archived hours
unwatched = df[df["Minutes Delivered"] == 0]['Archived Hours (Unedited hours)'].sum()
print('Unwatched archived hours:', round(unwatched))

Unwatched archived hours: 12484


In [6]:
# archived hours over four years old
# there is probably substantial overlap between this and the unwatched hours
import datetime
four_years_ago = datetime.datetime.now() - datetime.timedelta(days=4 * 365)
over_four_years_old = df[df["Start Time"] < four_years_ago]['Archived Hours (Unedited hours)'].sum()
print('Archived hours over four years old:', round(over_four_years_old))

Archived hours over four years old: 11233


In [12]:
# Unwatched, archived hours over four years old in Personal folders (under Users hierarchy)
personal_over_four_years_old = df[(df["Start Time"] < four_years_ago) &
    (df["Root Folder (Level 0)"] == "Users")
]["Archived Hours (Unedited hours)"].sum()
round(personal_over_four_years_old)

9941

In [14]:

# Unwatched, archived hours over four years old in Course folders (under Moodle hierarchy)
course_over_four_years_old = df[(df["Start Time"] < four_years_ago) &
    (df["Root Folder (Level 0)"] == "Moodle")
]["Archived Hours (Unedited hours)"].sum()
round(course_over_four_years_old)

1193

In [15]:
# All other unwatched, archived hours over four years old
print(round(over_four_years_old - personal_over_four_years_old - course_over_four_years_old))


99
